# Calculating the USD Liquidity Index with the OpenBB SDK

This popular indicator is made from a simple subtraction of three FRED series that are published every Wednesday, and is often overlayed with risk assets like the S&P 500 Index or Bitcoin.  The OpenBB SDK is well suited for this task, let's take a look to create this index.

The formula is defined as:

```console
WALCL (All Liabilities) – WLRRAL (RRP) – WDTGAL (TGA)
```

To get these data series, we will use the `fred` function in the `economy` module.  First thing is to import the SDK, and we will also import Pandas to conduct some DataFrame operations.

In [1]:
import pandas as pd
from openbb_terminal.sdk import openbb

In [21]:
data = openbb.economy.fred(["WALCL", "WLRRAL", "WDTGAL"])

## Querying FRED

If we didn't already know the ID for the series, we can search with:

```python
openbb.economy.fred_notes("Wednesday Levels")
```

In [24]:
# The first result is the series we are looking for as the starting value.

openbb.economy.fred_notes("Wednesday Levels").head(3)

,id,realtime_start,realtime_end,title,observation_start,observation_end,frequency,frequency_short,units,units_short,seasonal_adjustment,seasonal_adjustment_short,last_updated,popularity,group_popularity,notes
0,WALCL,2023-05-22,2023-05-22,Assets: Total Assets: Total Assets (Less\nElim...,2002-12-18,2023-05-17,"Weekly, As of Wednesday",W,Millions of U.S. Dollars,Mil. of U.S. $,Not Seasonally Adjusted,NSA,2023-05-18 15:34:02-05,97,97,No description provided.
1,TREAST,2023-05-22,2023-05-22,Assets: Securities Held Outright: U.S. Treasur...,2002-12-18,2023-05-17,"Weekly, As of Wednesday",W,Millions of U.S. Dollars,Mil. of U.S. $,Not Seasonally Adjusted,NSA,2023-05-18 15:35:02-05,73,73,The total face value of U.S. Treasury securiti...
2,WSHOMCB,2023-05-22,2023-05-22,Assets: Securities Held Outright: Mortgage-Bac...,2002-12-18,2023-05-17,"Weekly, As of Wednesday",W,Millions of U.S. Dollars,Mil. of U.S. $,Not Seasonally Adjusted,NSA,2023-05-18 15:35:04-05,72,72,The current face value of mortgage-backed obli...


In [25]:
# Adding "Reverse Repo" to the search returns the second series in the equation, as the first result.

openbb.economy.fred_notes("Wednesday Levels Reverse Repo").head(3)

,id,realtime_start,realtime_end,title,observation_start,observation_end,frequency,frequency_short,units,units_short,seasonal_adjustment,seasonal_adjustment_short,last_updated,popularity,group_popularity,notes
0,WLRRAL,2023-05-22,2023-05-22,Liabilities and Capital: Liabilities: Reverse\...,2002-12-18,2023-05-17,"Weekly, As of Wednesday",W,Millions of U.S. Dollars,Mil. of U.S. $,Not Seasonally Adjusted,NSA,2023-05-18 15:34:11-05,57,57,Reverse repurchase agreements are transactions...
1,WLRRAFOIAL,2023-05-22,2023-05-22,Liabilities and Capital: Liabilities: Reverse\...,2002-12-18,2023-05-17,"Weekly, As of Wednesday",W,Millions of U.S. Dollars,Mil. of U.S. $,Not Seasonally Adjusted,NSA,2023-05-18 15:34:14-05,43,43,Reverse repurchase agreements are transactions...
2,WLRRAOL,2023-05-22,2023-05-22,Liabilities and Capital: Liabilities: Reverse\...,2002-12-18,2023-05-17,"Weekly, As of Wednesday",W,Millions of U.S. Dollars,Mil. of U.S. $,Not Seasonally Adjusted,NSA,2023-05-18 15:34:17-05,20,20,No description provided.


In [26]:
# Refining the search for the Treasury General Account, returns the final series in the equation, as the first result.

openbb.economy.fred_notes("Wednesday Levels Treasury General").head(3)

,id,realtime_start,realtime_end,title,observation_start,observation_end,frequency,frequency_short,units,units_short,seasonal_adjustment,seasonal_adjustment_short,last_updated,popularity,group_popularity,notes
0,WDTGAL,2023-05-22,2023-05-22,Liabilities and Capital: Liabilities: Deposits...,2002-12-18,2023-05-17,"Weekly, As of Wednesday",W,Millions of U.S. Dollars,Mil. of U.S. $,Not Seasonally Adjusted,NSA,2023-05-18 15:34:08-05,68,68,This account is the primary operational accoun...
1,D2WLTGAL,2023-05-22,2023-05-22,Liabilities and Capital: Liabilities: Deposits...,2002-12-18,2023-05-17,"Weekly, As of Wednesday",W,Millions of U.S. Dollars,Mil. of U.S. $,Not Seasonally Adjusted,NSA,2023-05-18 15:34:12-05,62,62,No description provided.
2,WLDLCL,2023-05-22,2023-05-22,Liabilities and Capital: Liabilities: Deposits...,2002-12-18,2023-05-17,"Weekly, As of Wednesday",W,Millions of U.S. Dollars,Mil. of U.S. $,Not Seasonally Adjusted,NSA,2023-05-18 15:34:18-05,26,26,"This item is the sum of ""Term deposits held by..."


Let's inspect the object that was returned containing all three series.  Notice that the object is a Tuple, and it contains the time series data as the first element and the second element is a dictionary of meta data.  By looking at the meta data, we can confirm that all three series are numbers as `Millions of USD`.  If they were not all equivalent, some adjustments would need to be made before applying the equation.

In [28]:
data[1]

{'WALCL': {'title': 'Assets: Total Assets: Total Assets (Less Eliminations from Consolidation): Wednesday Level',
  'units': 'Mil. of U.S. $'},
 'WLRRAL': {'title': 'Liabilities and Capital: Liabilities: Reverse Repurchase Agreements: Wednesday Level',
  'units': 'Mil. of U.S. $'},
 'WDTGAL': {'title': 'Liabilities and Capital: Liabilities: Deposits with F.R. Banks, Other Than Reserve Balances: U.S. Treasury, General Account: Wednesday Level',
  'units': 'Mil. of U.S. $'}}

In [29]:
# Inspecting the time series element shows that the three series all start and end on the same date, so they will be easy to work with.

data[0]

,WALCL,WLRRAL,WDTGAL
2002-12-18,719542.0,21905.0,6595.0
2002-12-25,732059.0,20396.0,4662.0
2003-01-01,730994.0,21091.0,4420.0
2003-01-08,723762.0,18709.0,5490.0
2003-01-15,720074.0,17813.0,5192.0
...,...,...,...
2023-04-19,8593263.0,2666343.0,265095.0
2023-04-26,8562768.0,2638805.0,296209.0
2023-05-03,8503994.0,2640951.0,188309.0
2023-05-10,8503017.0,2618178.0,154808.0


Applying the formula will simply be a matter of subtracting the three columns.

In [31]:
liquidity_index = pd.DataFrame(data[0])
liquidity_index["USD Liquidity Index"] = (liquidity_index["WALCL"] - liquidity_index["WLRRAL"] - liquidity_index["WDTGAL"])

In [33]:
liquidity_index

,WALCL,WLRRAL,WDTGAL,USD Liquidity Index
2002-12-18,719542.0,21905.0,6595.0,691042.0
2002-12-25,732059.0,20396.0,4662.0,707001.0
2003-01-01,730994.0,21091.0,4420.0,705483.0
2003-01-08,723762.0,18709.0,5490.0,699563.0
2003-01-15,720074.0,17813.0,5192.0,697069.0
...,...,...,...,...
2023-04-19,8593263.0,2666343.0,265095.0,5661825.0
2023-04-26,8562768.0,2638805.0,296209.0,5627754.0
2023-05-03,8503994.0,2640951.0,188309.0,5674734.0
2023-05-10,8503017.0,2618178.0,154808.0,5730031.0


To overlay the calculated result with a risk asset, we will need to load some more data.  Let's get the S&P 500 index levels.

In [37]:
sp500 = openbb.economy.index(["^GSPC"], start_date = '2002-12-18').rename(columns = {"^GSPC": "S&P 500"})

In [39]:
sp500.tail(3)

,S&P 500
Date,
2023-05-17,4158.770020
2023-05-18,4198.049805
2023-05-19,4191.979980


This data is daily, but joining the two DataFrames with the weekly date index on the left merges only the values corresponding to those dates.  Performing this operation shows that `NaN` values are present where the Wednesday was Christmas and New Years Day.

In [42]:
liquidity_index.join(sp500)

,WALCL,WLRRAL,WDTGAL,USD Liquidity Index,S&P 500
2002-12-18,719542.0,21905.0,6595.0,691042.0,891.119995
2002-12-25,732059.0,20396.0,4662.0,707001.0,NaN
2003-01-01,730994.0,21091.0,4420.0,705483.0,NaN
2003-01-08,723762.0,18709.0,5490.0,699563.0,909.929993
2003-01-15,720074.0,17813.0,5192.0,697069.0,918.219971
...,...,...,...,...,...
2023-04-19,8593263.0,2666343.0,265095.0,5661825.0,4154.520020
2023-04-26,8562768.0,2638805.0,296209.0,5627754.0,4055.989990
2023-05-03,8503994.0,2640951.0,188309.0,5674734.0,4090.750000
2023-05-10,8503017.0,2618178.0,154808.0,5730031.0,4137.640137


To solve this little problem, use `fillna` and apply the `ffill` method.

In [45]:
liquidity_index = liquidity_index.join(sp500).fillna(method = 'ffill')

liquidity_index

,WALCL,WLRRAL,WDTGAL,USD Liquidity Index,S&P 500
2002-12-18,719542.0,21905.0,6595.0,691042.0,891.119995
2002-12-25,732059.0,20396.0,4662.0,707001.0,891.119995
2003-01-01,730994.0,21091.0,4420.0,705483.0,891.119995
2003-01-08,723762.0,18709.0,5490.0,699563.0,909.929993
2003-01-15,720074.0,17813.0,5192.0,697069.0,918.219971
...,...,...,...,...,...
2023-04-19,8593263.0,2666343.0,265095.0,5661825.0,4154.520020
2023-04-26,8562768.0,2638805.0,296209.0,5627754.0,4055.989990
2023-05-03,8503994.0,2640951.0,188309.0,5674734.0,4090.750000
2023-05-10,8503017.0,2618178.0,154808.0,5730031.0,4137.640137


Now that there are two items to compare, let's draw it!

In [49]:
# Import a Terminal function for plotting with two y-axis.  Each axis will require its own DataFrame.

from openbb_terminal.economy.plot_view import show_plot

y_axis1 = pd.DataFrame(liquidity_index[["USD Liquidity Index"]])
y_axis2 = pd.DataFrame(liquidity_index[["S&P 500"]])

In [50]:
show_plot(y_axis1, y_axis2)

To draw them both on the same y-axis, they will need to be normalized.

In [68]:
y_axis = liquidity_index[["USD Liquidity Index", "S&P 500"]]

def absolute_maximum_scale(series):
    return series / series.abs().max()

def min_max_scaling(series):
    return (series - series.min()) / (series.max() - series.min())

def z_score_standardization(series):
    return (series - series.mean()) / series.std()

methods = {"z": z_score_standardization, "m": min_max_scaling, "a": absolute_maximum_scale}

def normalize(data: pd.DataFrame, method: str = "z") -> pd.DataFrame: 
    for col in data.columns:
        data[col] = methods[f"{method}"](data[col])

    return data


In [64]:
normalized = normalize(y_axis, method = "m")

normalized.tail(3)

,USD Liquidity Index,S&P 500
2023-05-03,0.818334,0.827873
2023-05-10,0.827407,0.839365
2023-05-17,0.837214,0.844544


We can use `openbb.forecast.plot` to make a fast chart of multiple columns on the same y-axis.

In [71]:
openbb.forecast.plot(normalize(y_axis, method = "m"), columns = normalized.columns)

By adding, `external_axes = True`, the plot is returned as a Plotly Figure object.  This allows the chart to be updated and further refined.

In [77]:
fig = openbb.forecast.plot(normalize(y_axis, method = "m"), columns = normalized.columns, external_axes = True)

fig.update({'layout': 
    {'yaxis': {'title': 'Min/Max Normalized Values'},
    'title': 'USD Liquidity Index vs. S&P 500',
}})
fig.update_layout(
    {
    'title_y':0.98,
    'title_x':0.5,
    },
    legend=dict(
        yanchor="top",
        y=1,
        xanchor="left",
        x=0
    ),
    width = 1200,
    height = 500,
)

The combinations are endless and we love seeing your creations, tag us on social media with your custom indexes and indicators.